In [12]:
import numpy as np
import trimesh
import pyvista as pv
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

from bfieldtools.mesh_conductor import MeshConductor
from bfieldtools.coil_optimize import optimize_streamfunctions
from bfieldtools.contour import scalar_contour
from bfieldtools.viz import plot_3d_current_loops, plot_data_on_vertices

import pkg_resources

# Set the notebook plotting backend for PyVista
pv.set_jupyter_backend('trame')

# Set unit, e.g. meter or millimeter.
# This doesn't matter, the problem is scale-invariant
scaling_factor = 1 # meter

# Load example coil mesh that is centered on the origin
coilmesh = trimesh.load(
    file_obj=pkg_resources.resource_filename(
        "bfieldtools", "example_meshes/open_cylinder.stl"
    ),
    process=True,
)

angle = np.pi / 2
rotation_matrix = np.array(
    [
        [np.cos(angle), 0, np.sin(angle), 0],
        [0, 1, 0, 0],
        [-np.sin(angle), 0, np.cos(angle), 0],
        [0, 0, 0, 1],
    ]
)

coilmesh1 = coilmesh.copy()

# Define the new radius and height
new_radius_scale = .5  # Scale factor for the radius
new_height_scale = 0.25  # Scale factor for the height

# Adjust the coil mesh vertices for the new radius and height
coilmesh1.vertices[:, :2] *= new_radius_scale  # Scale x and y coordinates for radius
coilmesh1.vertices[:, 2] *= new_height_scale   # Scale z coordinate for height

coilmesh1.apply_transform(rotation_matrix)

# Proceed with creating the MeshConductor objects with the updated meshes
coil = MeshConductor(
    verts=coilmesh1.vertices,
    tris=coilmesh1.faces,
    fix_normals=True,
    basis_name="suh",
    N_suh=400,
)

def alu_sigma(T):
    ref_T = 293  # K
    ref_rho = 2.82e-8  # ohm*meter
    alpha = 0.0039  # 1/K

    rho = alpha * (T - ref_T) * ref_rho + ref_rho

    return 1 / rho

resistivity = 1 / alu_sigma(T=293)  # room-temp Aluminium
thickness = 0.5e-3  # 0.5 mm thick  

# Set up target points and plot geometry
center = np.array([0, 0, 0])
sidelength = 0.1 * scaling_factor # 0.1 meter
n = 8
xx = np.linspace(-sidelength / 2, sidelength / 2, n)
yy = np.linspace(-sidelength / 2, sidelength / 2, n)
zz = np.linspace(-sidelength / 2, sidelength / 2, n)
X, Y, Z = np.meshgrid(xx, yy, zz, indexing="ij")

x = X.ravel()
y = Y.ravel()
z = Z.ravel()

target_points = np.array([x, y, z]).T

# Turn cube into sphere by rejecting points "in the corners"
target_points = (
    target_points[np.linalg.norm(target_points, axis=1) < sidelength / 2] + center
)

# Define the undesired magnetic field that you want to cancel
# Example: cancel magnetic field in all three directions
undesired_field = np.zeros(target_points.shape)
undesired_field[:, 0] = 1  # Example non-uniform field in x-direction
undesired_field[:, 1] = 1  # Example non-uniform field in y-direction
undesired_field[:, 2] = 1  # Example non-uniform field in z-direction

# The target field to cancel the undesired field
target_field = -undesired_field

# Create bfield specifications used when optimizing the coil geometry
target_spec = {
    "coupling": coil.B_coupling(target_points),
    "abs_error": 0.01,
    "target": target_field,
}

from scipy.linalg import eigh

# Run QP solver to optimize stream function
import mosek

try:
    coil.s, prob = optimize_streamfunctions(
        coil,
        [target_spec],
        objective="minimum_inductive_energy",
        solver="MOSEK",
        solver_opts={"mosek_params": {mosek.iparam.num_threads: 8}},
    )
except Exception as e:
    print(f"Optimization failed: {e}")
    coil.s = None

if coil.s is not None:
    from bfieldtools.mesh_conductor import StreamFunction

    # Plot coil windings and target points
    loops = scalar_contour(coil.mesh, coil.s.vert, N_contours=15)
    # Convert loops to a single polydata object
    loops_polydata = pv.PolyData()
    loops_polydata.points = np.vstack(loops)
    loops_polydata.faces = np.hstack([np.full(len(loop), 3) for loop in loops] + [np.arange(len(loops[0]), len(loops_polydata.points))])

 
    # Interactive plotting with PyVista and ipywidgets
    plotter = pv.Plotter(notebook=True)
    plotter.set_background("white")
    pv.global_theme.font.color = "black"

    meshviz_coil = pv.PolyData(
        coil.mesh.vertices * 0.75, np.hstack((np.repeat(3, len(coil.mesh.faces))[:, None], coil.mesh.faces))
    )
    plotter.add_mesh(meshviz_coil, opacity=0.2)

    # Add target points
    plotter.add_points(target_points, color='red', point_size=5)

    plotter.add_mesh(loops_polydata, color="red", line_width=5)
    # Add coil windings
    # for loop in loops:
    #     plotter.add_lines(loop, color='black', width=1)

    # Function to update arrows based on current in mA
    def update_arrows(current_mA):
        plotter.clear_actors()
        plotter.add_mesh(meshviz_coil, opacity=0.2)
        plotter.add_points(target_points, color='red', point_size=5)
        plotter.add_mesh(loops_polydata, color="red", line_width=5)
        # for loop in loops:
        #     plotter.add_lines(loop, color='black', width=1)
        B_target = coil.B_coupling(target_points) @ (coil.s * current_mA / 1000)  # Convert mA to A
        plotter.add_arrows(target_points, B_target, mag=0.1, color='blue')
        plotter.show()

    # Create the interactive widget
    current_slider = widgets.FloatSlider(
        value=100.0,
        min=10.0,
        max=500.0,
        step=10.0,
        description='Current (mA):',
        continuous_update=False
    )

    # Display the interactive widget
    interact(update_arrows, current_mA=current_slider)
else:
    print("Optimization did not return valid results. Please check your inputs and solver setup.")


Calculating surface harmonics expansion...
Computing the laplacian matrix...
Computing the mass matrix...
Computing magnetic field coupling matrix, 4764 vertices by 160 target points... took 0.75 seconds.
Computing the inductance matrix...
Computing self-inductance matrix using rough quadrature (degree=2).              For higher accuracy, set quad_degree to 4 or more.
Estimating 69923 MiB required for 4764 by 4764 vertices...
Computing inductance matrix in 240 chunks (5859 MiB memory free),              when approx_far=True using more chunks is faster...
Computing triangle-coupling matrix
Inductance matrix computation took 57.11 seconds.
Passing problem to solver...
                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) Jun 05 01:35:57 PM: Your problem has 400 variables, 960 constraints, and 0 parameters.
(CVXPY) Jun 05 01:35:57 PM: It is compliant with the following gramma

: 